# 07 - Model 2 Training

In [ ]:
# importing libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, UpSampling2D, concatenate, LeakyReLU, ReLU
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.metrics import MeanIoU
from sklearn.model_selection import train_test_split
import tifffile
from pathlib import Path
import os
import mlflow
import mlflow.tensorflow
from tqdm import tqdm
import time

# Project path
TERRAFLOOD = Path('../')

In [ ]:
# ------------ inputs --------------
# experiment's meta data
experiment_number = 1
model_architectre = 2
epochs = 200
early_stopping_patience = 10
learning_rate_patience = 5

# input and output paths
load_path = TERRAFLOOD.joinpath("dataset_balanced/")
save_path = TERRAFLOOD.joinpath(f"experiments/model_{model_architecture}_exp_{experiment_number}/")

# structure of logging and saving with naming convention
# Checkpoint of the model
checkpoint_dir = save_path.joinpath("checkpoint/")
checkpoint_path = checkpoint_dir / f"model_{model_architecture}_check_exp_{experiment_number}_epochs_{epochs}_patience_{early_stopping_patience}_{learning_rate_patience}.keras"

# Logging on tensorboard
tensorboard_logs_dir = save_path.joinpath("tensorboard_log/")

# Saving final log on ML-Flow
mlflow_logs_dir = save_path.joinpath("mlflow_log/")

# Saving the final model
model_save_dir = save_path.joinpath("model/")
model_path = model_save_dir / f"model_{model_architecture}_exp_{experiment_number}_epochs_{epochs}_patience_{early_stopping_patience}_{learning_rate_patience}.keras"

# directory existence ensurance
checkpoint_dir.mkdir(parents=True, exist_ok=True)
tensorboard_logs_dir.mkdir(parents=True, exist_ok=True)
mlflow_logs_dir.mkdir(parents=True, exist_ok=True)
model_save_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
# Hardware info
# Print TensorFlow version
print("TensorFlow version:", tf.__version__)

# List available devices
devices = tf.config.list_physical_devices()
print("Available devices:", devices)

# GPU info in details (assuming nvidia as GPU device)
!nvidia-smi

In [ ]:
# Define Jaccard Loss
def jaccard_loss(y_true, y_pred, smooth=100):
    """
    Calculates the Jaccard loss, also known as the Intersection over Union (IoU) loss.
    Args:
        y_true (tensor): Ground truth labels.
        y_pred (tensor): Predicted labels.
        smooth (float): Smoothing factor to avoid division by zero.
    Returns:
        jaccard loss (tensor)
    """
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    sum_ = tf.keras.backend.sum(y_true_f + y_pred_f)
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return 1 - jac

In [ ]:
# Define U-Net model with 2 input channels
def unet_two_channels(input_size=(256, 256, 2), loss_function='jaccard_loss'):
    inputs = Input(input_size)

    def conv_block(inputs, num_filters):
        conv = Conv2D(num_filters, 3, activation=LeakyReLU(), padding="same")(inputs)
        conv = Conv2D(num_filters, 3, activation=LeakyReLU(), padding="same")(conv)
        return conv

    conv1 = conv_block(inputs, 64)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = conv_block(pool1, 128)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = conv_block(pool2, 256)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = conv_block(pool3, 512)
    drop4 = Dropout(0.3)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = conv_block(pool4, 1024)
    drop5 = Dropout(0.3)(conv5)

    def up_block(inputs, skip_connection, num_filters):
        up = Conv2D(num_filters, 2, activation=LeakyReLU(), padding="same")(UpSampling2D(size=(2, 2))(inputs))
        merge = concatenate([skip_connection, up], axis=3)
        conv = conv_block(merge, num_filters)
        return conv

    conv6 = up_block(drop5, drop4, 512)
    conv7 = up_block(conv6, conv3, 256)
    conv8 = up_block(conv7, conv2, 128)
    conv9 = up_block(conv8, conv1, 64)

    conv9 = Conv2D(2, 1, activation=LeakyReLU(), padding="same")(conv9)
    conv10 = Conv2D(1, 1, activation="sigmoid")(conv9)

    model = Model(inputs, conv10)

    model.compile(optimizer=RMSprop(learning_rate=1e-4), loss=jaccard_loss, metrics=["accuracy", MeanIoU(num_classes=2)])
    return model
